In [1]:
from datetime import datetime
import time

from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

In [2]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
date_fr = "11/01/2019"
date_to = "10/31/2020"

In [4]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [5]:
start_time = time.time()

In [ ]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 1)
    get_bookings = booking_data.get()

    df = pipeline.fit_transform(get_bookings)

    if df is not None:
        df.to_csv(f"{destination}_OUT.csv", index=False)
        stmt = insert(reservation).values(df.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)

In [ ]:
elapsed_time = time.time() - start_time
elapsed_time